# Monthly wages for females and males
In LOCAL CURRENCY UNITS

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [2]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
files = ['MONWAGFE.csv', 'MONWAGMA.csv']
dfs = []
for f in files:
    df = pd.read_csv(f)
    dfs.append((f, df))    

In [3]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [4]:
# import US specific data (median dollar earnings per month)
us_data = pd.read_csv("median_US_earnings.csv")
us_data.head()  

,year,female,male
0,1995,3133,4386
1,1996,3216,4360
2,1997,3316,4471
3,1998,3388,4630
4,1999,3376,4668


In [5]:
wages_fem = dfs[0][1].iloc[0:85, :]  # monthly wages for females
wages_m = dfs[1][1].iloc[0:85, :] # monthly wages for males
# wages_fem = wages_fem.iloc[0:85, :]  # rm bottom descrip rows
# wages_m = wages_m.iloc[0:85, :]  # rm bottom descrip rows
wages_fem_orig = wages_fem.copy()
wages_m_orig =wages_m.copy()
# wages_fem = wages_fem[~wages_fem['2011'].isna()]
# wages_m = wages_m[~wages_m['2011'].isna()]
print(wages_fem.shape, wages_m.shape) # (26,18) -> we have 26 countries


(85, 18) (85, 18)


In [6]:
wages_fem.iloc[:, 1:] = wages_fem.iloc[:, 1:].fillna(axis=1, method='pad')
wages_m.iloc[:, 1:] = wages_m.iloc[:, 1:].fillna(axis=1, method='pad')
wages_fem.iloc[:, 1:] = wages_fem.iloc[:, 1:].fillna(axis=1, method='bfill')
wages_m.iloc[:, 1:] = wages_m.iloc[:, 1:].fillna(axis=1, method='bfill')

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [7]:
print(wages_fem.isna().sum().sum())
print(wages_m.isna().sum().sum())

0
0


In [8]:
# wages_fem_orig.head()
# wages_fem.head()

In [9]:
# process us data separartely
us_data2 = us_data.transpose()
us_data2.columns = us_data2.iloc[0,:]
print(us_data2.isna().sum().sum())
wages_fem_us = pd.DataFrame(us_data2.iloc[1,:])
wages_m_us = pd.DataFrame(us_data2.iloc[2,:])


0


In [10]:
wages_m_us.rename(columns={'male': 'value'}, inplace=True)
wages_fem_us.rename(columns={'female': 'value'}, inplace=True)


In [11]:
wages_fem_us['gender'] = 'female'
wages_fem_us.reset_index(inplace=True)
wages_m_us['gender'] = 'male'
wages_m_us.reset_index(inplace=True)

In [12]:
wages_us = pd.concat([wages_fem_us, wages_m_us], axis=0)
wages_us['ISO'] = 'USA'
wages_us2 = wages_us[wages_us['year'] != 'ISO']
# wages_us2.drop(labels='index', axis=1, inplace=True)

In [13]:
## combine BOTH genders one dataset for ONE country
""" 
1. Narrow down country
2. Separately for each M / F dataset, melt the dataset
3. concat the melted datasets together
"""
all_countries = pd.DataFrame()
countries = list(wages_fem['ISO'].unique())
countries.remove("MLT")

## Choose a country
for c in countries:
    f1 = wages_fem[wages_fem['ISO'] == c]
    f2 = f1.melt().iloc[1:, :]
    f2.columns = ['year', 'value']
    f2['gender'] = 'female'
    m1 = wages_m[wages_m['ISO'] == c]
    m2 = m1.melt().iloc[1:, :]
    m2.columns = ['year', 'value']
    m2['gender'] = 'male'
    all = pd.concat([f2, m2], axis=0)
    all2 = all[all['year'] != 'gender']
    all2['ISO'] = c
    all_countries = pd.concat([all_countries, all2], axis=0)


In [14]:
# all_countries.head()
print(all_countries.shape)

(2856, 4)


In [15]:
# all_countries.reset_index(inplace=True)
# wages_us2.reset_index(inplace=True)
all_countries2 = pd.concat([all_countries, wages_us2], axis=0)
print(all_countries2.shape)

(2910, 4)


In [16]:
df = all_countries2
df.dropna(axis=0, inplace=True)

In [17]:
## check for missing values
# df.isna().sum().sort_values(ascending=False)
# df.drop(labels='index', axis=1, inplace=True)

In [18]:
df.shape

(2910, 4)

In [19]:
df.head()

,year,value,gender,ISO
1,1995,480.47,female,ARG
2,1996,480.47,female,ARG
3,1997,485.53,female,ARG
4,1998,509.2,female,ARG
5,1999,504.33,female,ARG


In [20]:
## Map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
country_names_dict = { v: k for k, v in country_names.items()}
def convert_iso_to_name(iso):
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso
# df['country'] = df['ISO']
df['country'] = df['ISO'].apply(convert_iso_to_name)
# df.drop(labels='ISO', axis=1, inplace=True)

In [21]:
df['value'] = df['value'].astype(int)

In [22]:
df = df[['year', 'country', 'value', 'gender']]

In [23]:
print(df.shape)

(2910, 4)


In [24]:
df.head()

,year,country,value,gender
1,1995,Argentina,480,female
2,1996,Argentina,480,female
3,1997,Argentina,485,female
4,1998,Argentina,509,female
5,1999,Argentina,504,female


In [26]:
# df['country'].nunique() # 85 countries

In [ ]:
## TO EXPORT AS CSV
# df.to_csv("wages.csv", index=False)

# Reformat for Overview

In [ ]:
## put all lines on one chart
## my concern: 85 countries, 2 gender lines, thats too many lines on a cahrt. its not going to be readable
# TODO: organize by continent?
# TODO: calc acutal gap as wage difference, instead of male and fem wages on separate lines

